# Immigration and Occupation Convergence in the Early 1900s

In this project, we gain experience manipulating and running regressions on "big data" in economic history. We use the USA IPUMS decennial census records from 1900, 1910, and 1920 and Census Tree Project crosswalks to link individuals across time. 

Our goal is to examine how immigration in the early 1900s affected income in the United States and determine whether the incomes immigrants started low and converged to the native-born population over time, the popular idea at the time. We will use three modeling techniques from (i) simple cross sections (reflecting the common view at the time) to (ii) repeated cross sections with cohort fixed effects (addressing the changing occupation distributions of immigrants in different periods) and (iii) linked panel data (controlling for individual characteristics). 

Our methods closely follow the work of Abramitzky, Boustan, and Eriksson (2014) in their paper "A Nation of Immigrants: Assimilation and Economic Outcomes in the Age of Mass Migration" but with more complete data (that was not available at the time) and complete analytical code.

In [1]:
import pandas as pd
import statsmodels.formula.api as smf


data_path = "data/"

samples_1900_dat = data_path + "ipums_1900.dat"
samples_1910_dat = data_path + "ipums_1910.dat"
samples_1920_dat = data_path + "ipums_1920.dat"

links00to10_csv = data_path + "census_links_1900_1910.csv"
links10to20_csv = data_path + "census_links_1910_1920.csv"


out_path = "out/"

linked_data_csv = out_path + "combined_linked_data.csv"

model1_data_csv = out_path + "cross_section_data.csv"
model1_data_male_csv = out_path + "cross_section_data_male.csv"
model1_data_female_csv = out_path + "cross_section_data_female.csv"

model2_data_csv = out_path + "repeated_cross_section_data.csv"
model2_data_male_csv = out_path + "repeated_cross_section_data_male.csv"
model2_data_female_csv = out_path + "repeated_cross_section_data_female.csv"

model3_data_csv = out_path + "long_term_panel_data.csv"
model3_data_male_csv = out_path + "long_term_panel_data_male.csv"
model3_data_female_csv = out_path + "long_term_panel_data_female.csv"


# Historic CPI data from
# https://www.minneapolisfed.org/about-us/monetary-policy/inflation-calculator/consumer-price-index-1913-
CPI_1950 = 24.1
CPI_2010 = 218.1

# Data Processing

We make 3 separate IPUMS extractions for 1900, 1910, and 1920. For each extraction, we select the following 16 variables. More details on these variables can be viewed in the corresponding "basic_desc" codebooks (also within the data folder).
1. YEAR
2. SEX
3. AGE
4. RACE
5. RACED
6. BPL
7. BPLD
8. NATIVITY
9. YRIMMIG
10. OCC1950
11. OCCSCORE
12. HISTID
13. HIK
14. LINK1900
15. LINK1910
16. LINK1920

When requesting these extractions from IPUMS, we only select rows (individuals) that meet the following cases:
1. For 1900 data, AGE $\in [18, 35]$. For 1910, AGE $\in [28, 45]$. For 1920, AGE $\in [38, 55]$. This is to ensure individuals are likely able to be in the workforce across all three periods.
2. OCC1950 $\neq 997 \ or \ 999$. This filters out missing and unknown occupation codes.
3. LINK1900, LINK1910, and LINK1920 all equal 1. This ensures individuals can be linked across time.

We then process each DataFrame in the following way once we have read in the data:
1. Remove unnecessary columns YEAR, OCC1950, HIK, LINK1900, LINK1910, LINK1920.
2. Drop any rows with missing values.
3. Filter out immigrants who immigrated to the US outside of the 1880-1920 period.
4. Inflate the OCCSCORE (based in 1950) to 2010 dollars using the Consumer Price Index (CPI).
5. Add column for BIRTHCOUNTRY that consolidates different state codes of BPL to single US country code 1.
6. Add column IMMIGRANT that is 1 if the individual is foreign-born and 0 if otherwise.
7. Add categorical columns YRSTAY_`category` grouping immigrants by the length of time they've been in the US. Native-borns have none of these columns $\neq 0$.

Finally,  we use the Census Tree Project crosswalks to link individuals across time. We merge the data into a single dataset and clean the data to remove individuals with missing data.

In [2]:
# Reads in the IPUMS data from the provided `.dat` file given the expected format
def read_ipums_dat(dat_file, ):

    # Define column positions based on the provided format
    colspecs = [
        (0, 4),    # YEAR: Columns 1-4 (Length: 4)
        (4, 5),    # SEX: Column 5 (Length: 1)
        (5, 8),    # AGE: Columns 6-8 (Length: 3)
        (8, 9),    # RACE: Column 9 (Length: 1)
        (9, 12),   # RACED: Columns 10-12 (Length: 3)
        (12, 15),  # BPL: Columns 13-15 (Length: 3)
        (15, 20),  # BPLD: Columns 16-20 (Length: 5)
        (20, 21),  # NATIVITY: Column 21 (Length: 1)
        (21, 25),  # YRIMMIG: Columns 22-25 (Length: 4)
        (25, 28),  # OCC1950: Columns 26-28 (Length: 3)
        (28, 30),  # OCCSCORE: Columns 29-30 (Length: 2)
        (30, 66),  # HISTID: Columns 31-66 (Length: 36)
        (66, 87),  # HIK: Columns 67-87 (Length: 21)
        (87, 88),  # LINK1900: Column 88 (Length: 1)
        (88, 89),  # LINK1910: Column 89 (Length: 1)
        (89, 90)   # LINK1920: Column 90 (Length: 1)
    ]

    # Define corresponding column names
    column_names = [
        "YEAR", "SEX", "AGE", "RACE", "RACED", "BPL", "BPLD", "NATIVITY", "YRIMMIG",
        "OCC1950", "OCCSCORE", "HISTID", "HIK", "LINK1900", "LINK1910", "LINK1920"
    ]

    # Read the `.dat` file with fixed-width columns
    df = pd.read_fwf(dat_file, colspecs=colspecs, names=column_names)

    # Get the census year from the first row
    census_yr = df["YEAR"].iloc[0]

    # Drop columns that are not needed
    df.drop(columns=["YEAR", "OCC1950", "HIK", "LINK1900", "LINK1910", "LINK1920"], inplace=True)

    # Drop rows with missing values
    df.dropna(inplace=True)

    # Restrict to immigrants who arrived between 1880 and 1900
    df = df[df["YRIMMIG"].apply(lambda yr: yr == 0 or (yr >= 1880 and yr <= 1900))]

    # Inflate Occupational Income Score from 1950 to 2010 dollars
    df["OCCSCORE"] = df["OCCSCORE"] * (CPI_2010 / CPI_1950)

    # Add column for BIRTHCOUNTRY based on BPL
    # Set to 1 if code < 100 (different states), representing 1 (United States)
    # Note: US OUTLYING AREAS/TERRITORIES are considered foreign born
    df["BIRTHCOUNTRY"] = df["BPL"].apply(lambda code: 1 if code < 100 else code)

    # Add column for IMMIGRANT based on NATIVITY, bool 1 if foreign born
    df["IMMIGRANT"] = df["NATIVITY"].apply(lambda nativity: 1 if nativity == 5 else 0)

    # Defines main categorical variable grouping immigrants by length of stay in the US
    yr_stay_bins = [-1, 5, 10, 20, 30, 40] # 40 is max i.e. 1920 - 1880 = 40
    yr_stay_labels = ['0_5', '6_10', '11_20', '21_30', '30plus'] 
    # Note: Native born will have values = census_yr st year stay = NaN and no category is added.
    df['YRSTAY'] = pd.cut(census_yr - df['YRIMMIG'], bins=yr_stay_bins, labels=yr_stay_labels)
    df = pd.get_dummies(df, columns=['YRSTAY'])

    return df

df1900 = read_ipums_dat(samples_1900_dat)
df1910 = read_ipums_dat(samples_1910_dat)
df1920 = read_ipums_dat(samples_1920_dat)

In [3]:
# Visualizes the IPUMS data
print("IPUMS Data\n")

print("\nIPUMS 1900 Data has shape:", df1900.shape)
print(df1900.head())

print("\nIPUMS 1910 Data has shape:", df1910.shape)
print(df1910.head())

print("\nIPUMS 1920 Data has shape:", df1920.shape)
print(df1920.head())

IPUMS Data


IPUMS 1900 Data has shape: (2795961, 17)
   SEX  AGE  RACE  RACED  BPL  BPLD  NATIVITY  YRIMMIG    OCCSCORE  \
0    1   22     1    100    1   100         1        0  289.593361   
1    1   20     1    100    1   100         1        0    0.000000   
2    1   29     1    100    1   100         1        0  208.145228   
3    1   22     1    100    1   100         1        0  217.195021   
4    1   22     1    100    1   100         1        0  289.593361   

                                 HISTID  BIRTHCOUNTRY  IMMIGRANT  YRSTAY_0_5  \
0  D6A0F8D0-A265-4324-8DF6-85071C000BC5             1          0       False   
1  A1B3533A-18A2-47F3-A978-C4D5922F727F             1          0       False   
2  F3BEE221-26FC-4929-A397-23565CE4C6C4             1          0       False   
3  5DC163E6-7D72-4A24-82BD-936834804345             1          0       False   
4  51C1FD03-9250-4B29-A7B8-73F728DBE899             1          0       False   

   YRSTAY_6_10  YRSTAY_11_20  YRSTAY_21_30  

In [4]:
# Reads in the census links data from the provided `.csv` file
def read_census_links_csv(csv_file):
    # Read the `.csv` file
    df = pd.read_csv(csv_file)
    df = df.iloc[:, :2]
    return df

links00to10 = read_census_links_csv(links00to10_csv)
links10to20 = read_census_links_csv(links10to20_csv)

In [5]:
# Visualizes the Census Links data
print("Census Links Data\n")

print("\nCensus Links 1900 to 1910 Data has shape:", links00to10.shape)
print(links00to10.head())

print("\nCensus Links 1910 to 1920 Data has shape:", links10to20.shape)
print(links10to20.head())

Census Links Data


Census Links 1900 to 1910 Data has shape: (47369378, 2)
                             histid1900                            histid1910
0  0000002B-0B22-47D7-A357-06704078A544  20A57C40-CD53-4D1B-8A12-21B425FF1198
1  0000014C-B72A-4909-8730-1920F2E72550  DB9E3E38-4D68-4FFA-86ED-CA95DCF4019D
2  000001C9-2428-482B-BF88-13377AAACC86  2ED7151D-2905-45FE-99E8-E3FF9ED64A90
3  0000030C-CC2F-4FD1-A2AF-E2EB723EDF0E  8C714327-ED35-484F-A988-8B4C3819AB2D
4  000003AC-1740-4782-96DC-3C734D50C833  776A48B0-C56D-4DC9-AAFA-AF04FE8B10BE

Census Links 1910 to 1920 Data has shape: (57130269, 2)
                             histid1910                            histid1920
0  0000001C-C75E-4CE2-8497-19E3E48022AC  63E6C9CB-3351-45C2-85D3-DFD2432AF6BF
1  00000025-8BDE-45E4-BDDA-873F6AC6DD4C  F05DD254-0EEF-4452-BAE1-91116EB68FD5
2  000000A9-6115-4E1D-AA03-69B941FD8597  0B955257-A494-45F7-810B-9942EC2EF7DA
3  000000AF-BA85-4CDE-993C-7BC1F74952CE  AC99ADF5-4D9F-498B-8112-64A6415F6FE3
4  000000

# Linking the Data

The linking method uses the unique Historical Census IDs to match individuals across different census years. We first merge the 1900 and 1910 census data using the 1900 to 1910 Census Tree Project crosswalks. We then merge the resulting DataFrame with the 1920 census data using the 1910 to 1920 Census Tree Project crosswalks. This results in a DataFrame with individuals linked across all three census years.


In [6]:
# Merge the 1900 and 1910 data using census links 1900 to 1910
df00_10_merged = pd.merge(df1900, links00to10, left_on="HISTID", right_on="histid1900", how="inner")
df00_10_merged = pd.merge(df00_10_merged, df1910, left_on="histid1910", right_on="HISTID", how="inner", suffixes=("_1900", "_1910"))
df00_10_merged.drop(columns=["histid1900", "histid1910"], inplace=True)

print("\nMerged 1900 and 1910 Data has shape:", df00_10_merged.shape)
print(df00_10_merged.columns)
print(df00_10_merged.head())

# Merge the 1910 and 1920 data using census links 1910 to 1920
df = pd.merge(df00_10_merged, links10to20, left_on="HISTID_1910", right_on="histid1910", how="inner")
df = pd.merge(df, df1920.rename(columns=lambda x: x + "_1920"), left_on="histid1920", right_on="HISTID_1920", how="inner")
df.drop(columns=["histid1910", "histid1920"], inplace=True)

print("\nComplete 1900, 1910, 1920 Merged Panel Data has shape:", df.shape)
print(df.columns)
print(df.head())



Merged 1900 and 1910 Data has shape: (2435919, 34)
Index(['SEX_1900', 'AGE_1900', 'RACE_1900', 'RACED_1900', 'BPL_1900',
       'BPLD_1900', 'NATIVITY_1900', 'YRIMMIG_1900', 'OCCSCORE_1900',
       'HISTID_1900', 'BIRTHCOUNTRY_1900', 'IMMIGRANT_1900', 'YRSTAY_0_5_1900',
       'YRSTAY_6_10_1900', 'YRSTAY_11_20_1900', 'YRSTAY_21_30_1900',
       'YRSTAY_30plus_1900', 'SEX_1910', 'AGE_1910', 'RACE_1910', 'RACED_1910',
       'BPL_1910', 'BPLD_1910', 'NATIVITY_1910', 'YRIMMIG_1910',
       'OCCSCORE_1910', 'HISTID_1910', 'BIRTHCOUNTRY_1910', 'IMMIGRANT_1910',
       'YRSTAY_0_5_1910', 'YRSTAY_6_10_1910', 'YRSTAY_11_20_1910',
       'YRSTAY_21_30_1910', 'YRSTAY_30plus_1910'],
      dtype='object')
   SEX_1900  AGE_1900  RACE_1900  RACED_1900  BPL_1900  BPLD_1900  \
0         1        20          1         100         1        100   
1         1        29          1         100         1        100   
2         1        22          1         100         1        100   
3         1        2

# Modeling

We compare the occupational mobility of native-born and immigrant workers using the following equation.

$
\begin{equation}
    \text{Occupation score}_{ijmt} = \gamma_{t-m} + \mu_m + \theta_t + \alpha_j + \beta_1 \text{age}_{it} + \beta_2 \text{age}^2_{it} + \beta_3 \text{age}^3_{it} + \beta_4 \text{age}^4_{it} + \epsilon_{ijmt}
\end{equation}
$

Here, *i* represents the **individual**, *j* the **country of origin**, *m* the **year of arrival** in the United States, and *t* the **census year**. The difference, *t - m*, indicates the **number of years** an individual has spent in the US. Categorical indicator variables $\mu_m$, $\theta_t$, and $\alpha_j$ encode the fixed effects for **year of arrival**, **census year**, and **country of origin**, respectively.


The **occupation score** serves as an estimate of income, derived from the **labor market earnings** typically associated with a given occupation. This score remains **consistent within individuals** but can **vary between occupations**.  

The coefficients $\beta_1$ through $\beta_4$ describe how **labor market experience** influences an individual’s standing within the **occupational hierarchy**.  

A set of **indicator variables** $\gamma_{t-m}$ categorizes the **foreign-born population** based on the **length of time spent in the U.S.**. Native-born individuals serve as the omitted **reference group**. 

- **0–5 years**  
- **6–10 years**  
- **11–20 years**  
- **21–30 years**  
- **More than 30 years**  

The coefficient for the first dummy variable **0-5 years** reflects whether **immigrants experience an occupation-based earnings penalty or premium** upon first arriving in the US. Meanwhile, the coefficients for the remaining categories indicate whether immigrants eventually **narrow the earnings gap with natives or surpass them** in terms of occupation-based income.

In [7]:
# Demeans a dependent variable for the specified categorical variable
# i.e. subtracts the mean of the dependent variable for each category from the dependent variable
# Returns the demeaned dependent variable as a new column
# For in place modification, assign result to original column (df[dependent] = demean(df, dependent, category))
def demean(df, dependent, category):
    return df[dependent] - df.groupby(category)[dependent].transform("mean")

## Prepare the Model 1 Data

### Model 1: Simple cross-sections

Pooling 1900, 1910, 1920 samples with no $\mu_m$ cohort fixed effects.

For this model, we take all the pooled data of 1900, 1910, and 1920, demean by Census Year and Birth Country to encode the Census Year and Birth Country fixed effects. We then run the expected regression.


In [8]:
# Split the panel data into 3 different dataframes by census year and then recombine
df1900["YRCENSUS"] = 1900
df1910["YRCENSUS"] = 1910
df1920["YRCENSUS"] = 1920
df_cross = pd.concat([df1900, df1910, df1920])

# Demean OCCSCORE by YRCENSUS (Census Year Fixed Effects)
df_cross["OCCSCORE"] = demean(df_cross, "OCCSCORE", "YRCENSUS")

# Demean OCCSCORE by BIRTHCOUNTRY (Country of Origin Fixed Effects)
df_cross["OCCSCORE"] = demean(df_cross, "OCCSCORE", "BIRTHCOUNTRY")

# Split by sex 
df_cross_m = df_cross[df_cross["SEX"] == 1]
df_cross_f = df_cross[df_cross["SEX"] == 2]

In [9]:
# Save the cross-sectional data to a `.csv` file
df_cross.to_csv(model1_data_csv, index=False)
df_cross_m.to_csv(model1_data_male_csv, index=False)
df_cross_f.to_csv(model1_data_female_csv, index=False)

In [10]:
# Visualize the Main Model 1 data
print("Cross-Sectional Data for Model 1 has shape:", df_cross.shape)

print("\nCross-Sectional Data for Model 1 has columns:")
print(df_cross.columns)

print("\nCross-Sectional Data for Model 1 has summary statistics:")
print(df_cross.describe())

print("\nCross-Sectional Data for Model 1 has head:")
print(df_cross.head())

Cross-Sectional Data for Model 1 has shape: (8691158, 18)

Cross-Sectional Data for Model 1 has columns:
Index(['SEX', 'AGE', 'RACE', 'RACED', 'BPL', 'BPLD', 'NATIVITY', 'YRIMMIG',
       'OCCSCORE', 'HISTID', 'BIRTHCOUNTRY', 'IMMIGRANT', 'YRSTAY_0_5',
       'YRSTAY_6_10', 'YRSTAY_11_20', 'YRSTAY_21_30', 'YRSTAY_30plus',
       'YRCENSUS'],
      dtype='object')

Cross-Sectional Data for Model 1 has summary statistics:
                SEX           AGE          RACE         RACED           BPL  \
count  8.691158e+06  8.691158e+06  8.691158e+06  8.691158e+06  8.691158e+06   
mean   1.080861e+00  3.695469e+01  1.049524e+00  1.050228e+02  7.418004e+01   
std    2.726221e-01  9.530602e+00  2.185696e-01  2.217686e+01  1.240334e+02   
min    1.000000e+00  1.800000e+01  1.000000e+00  1.000000e+02  1.000000e+00   
25%    1.000000e+00  3.000000e+01  1.000000e+00  1.000000e+02  2.100000e+01   
50%    1.000000e+00  3.700000e+01  1.000000e+00  1.000000e+02  3.600000e+01   
75%    1.000000e+00  4.

## Prepare the Model 2 Data

### Model 2: Repeated cross-sections 

Pooling 1900, 1910, 1920 samples with $\mu_m$ cohort fixed effects.

This model is exactly the same as Model 1 but with Cohort fixed effects. We take all the pooled data of 1900, 1910, and 1920, demean by Census Year, Birth Country, and Immigration Year to encode the Census Year, Birth Country, and Cohort fixed effects. We then run the expected regression.

In [11]:
# Copy the cross-sectional data
df_rep_cross = df_cross.copy()

# Demean OCCSCORE by YRIMMIG (Cohort Fixed Effects)
df_rep_cross["OCCSCORE"] = demean(df_rep_cross, "OCCSCORE", "YRIMMIG")

# Split by sex
df_rep_cross_m = df_rep_cross[df_rep_cross["SEX"] == 1]
df_rep_cross_f = df_rep_cross[df_rep_cross["SEX"] == 2]

In [12]:
# Save the repeated cross-sectional data to a `.csv` file
df_rep_cross.to_csv(model2_data_csv, index=False)
df_rep_cross_m.to_csv(model2_data_male_csv, index=False)
df_rep_cross_f.to_csv(model2_data_female_csv, index=False)

In [13]:
# Visualize the Main Model 2 data
print("Repeated Cross-Sectional Data for Model 2 has shape:", df_rep_cross.shape)

print("\nRepeated Cross-Sectional Data for Model 2 has columns:")
print(df_rep_cross.columns)

print("\nRepeated Cross-Sectional Data for Model 2 has summary statistics:")
print(df_rep_cross.describe())

print("\nRepeated Cross-Sectional Data for Model 2 has head:")
print(df_rep_cross.head())

Repeated Cross-Sectional Data for Model 2 has shape: (8691158, 18)

Repeated Cross-Sectional Data for Model 2 has columns:
Index(['SEX', 'AGE', 'RACE', 'RACED', 'BPL', 'BPLD', 'NATIVITY', 'YRIMMIG',
       'OCCSCORE', 'HISTID', 'BIRTHCOUNTRY', 'IMMIGRANT', 'YRSTAY_0_5',
       'YRSTAY_6_10', 'YRSTAY_11_20', 'YRSTAY_21_30', 'YRSTAY_30plus',
       'YRCENSUS'],
      dtype='object')

Repeated Cross-Sectional Data for Model 2 has summary statistics:
                SEX           AGE          RACE         RACED           BPL  \
count  8.691158e+06  8.691158e+06  8.691158e+06  8.691158e+06  8.691158e+06   
mean   1.080861e+00  3.695469e+01  1.049524e+00  1.050228e+02  7.418004e+01   
std    2.726221e-01  9.530602e+00  2.185696e-01  2.217686e+01  1.240334e+02   
min    1.000000e+00  1.800000e+01  1.000000e+00  1.000000e+02  1.000000e+00   
25%    1.000000e+00  3.000000e+01  1.000000e+00  1.000000e+02  2.100000e+01   
50%    1.000000e+00  3.700000e+01  1.000000e+00  1.000000e+02  3.600000e+01

## Prepare the Model 3 Data

### Model 3: Linked panel data

Linked panel data controlling for individual characteristics.

For this model, we only consider individuals who can be linked across all three decennial census years. This controls for individual characteristics! We demean by Census Year, Birth Country, and Immigration Year to encode the Census Year, Birth Country, and Cohort fixed effects. We then run the expected regression.

In [14]:
# Split the panel data into 3 different dataframes by census year and then recombine
df_panel1900 = df.filter(like="_1900").rename(columns=lambda x: x.replace("_1900", ""))
df_panel1910 = df.filter(like="_1910").rename(columns=lambda x: x.replace("_1910", ""))
df_panel1920 = df.filter(like="_1920").rename(columns=lambda x: x.replace("_1920", ""))
df_panel1900["YRCENSUS"] = 1900
df_panel1910["YRCENSUS"] = 1910
df_panel1920["YRCENSUS"] = 1920
df_panel = pd.concat([df_panel1900, df_panel1910, df_panel1920])

# Demean OCCSCORE by YRCENSUS (Census Year Fixed Effects)
df_panel["OCCSCORE"] = demean(df_panel, "OCCSCORE", "YRCENSUS")

# Demean OCCSCORE by BIRTHCOUNTRY (Country of Origin Fixed Effects)
df_panel["OCCSCORE"] = demean(df_panel, "OCCSCORE", "BIRTHCOUNTRY")

# Demean OCCSCORE by YRIMMIG (Cohort Fixed Effects)
df_panel["OCCSCORE"] = demean(df_panel, "OCCSCORE", "YRIMMIG")

# Split by SEX
df_panel_m = df_panel[df_panel["SEX"] == 1]
df_panel_f = df_panel[df_panel["SEX"] == 2]

In [15]:
# Save the panel data to a `.csv` file
df_panel.to_csv(model3_data_csv, index=False)
df_panel_m.to_csv(model3_data_male_csv, index=False)
df_panel_f.to_csv(model3_data_female_csv, index=False)

In [16]:
# Visualize the Main Model 3 data
print("Long-Term Panel Data for Model 3 has shape:", df_panel.shape)

print("\nLong-Term Panel Data for Model 3 has columns:")
print(df_panel.columns)

print("\nLong-Term Panel Data for Model 3 has summary statistics:")
print(df_panel.describe())

print("\nLong-Term Panel Data for Model 3 has head:")
print(df_panel.head())

Long-Term Panel Data for Model 3 has shape: (6686493, 18)

Long-Term Panel Data for Model 3 has columns:
Index(['SEX', 'AGE', 'RACE', 'RACED', 'BPL', 'BPLD', 'NATIVITY', 'YRIMMIG',
       'OCCSCORE', 'HISTID', 'BIRTHCOUNTRY', 'IMMIGRANT', 'YRSTAY_0_5',
       'YRSTAY_6_10', 'YRSTAY_11_20', 'YRSTAY_21_30', 'YRSTAY_30plus',
       'YRCENSUS'],
      dtype='object')

Long-Term Panel Data for Model 3 has summary statistics:
                SEX           AGE          RACE         RACED           BPL  \
count  6.686493e+06  6.686493e+06  6.686493e+06  6.686493e+06  6.686493e+06   
mean   1.016324e+00  3.685552e+01  1.041222e+00  1.041768e+02  6.803019e+01   
std    1.267176e-01  9.342978e+00  2.000961e-01  2.028285e+01  1.160994e+02   
min    1.000000e+00  1.800000e+01  1.000000e+00  1.000000e+02  1.000000e+00   
25%    1.000000e+00  3.000000e+01  1.000000e+00  1.000000e+02  2.000000e+01   
50%    1.000000e+00  3.700000e+01  1.000000e+00  1.000000e+02  3.600000e+01   
75%    1.000000e+00  4.

# Regression Results

In [17]:
# Run the regression model on the given data and print the results.
def regress_model(df):
    model = smf.ols("OCCSCORE ~ YRSTAY_0_5 + YRSTAY_6_10 + YRSTAY_11_20 + YRSTAY_21_30 + YRSTAY_30plus + AGE + I(AGE**2) + I(AGE**3) + I(AGE**4)", data=df).fit()
    print(model.summary())

## Model 1: Cross-Section Model
### Pooling 1900, 1910, 1920 samples with no $\mu_m$ cohort fixed effects.

In [18]:
print("Model 1: Cross-Sectional Data")
regress_model(df_cross)

Model 1: Cross-Sectional Data
                            OLS Regression Results                            
Dep. Variable:               OCCSCORE   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     8519.
Date:                Tue, 04 Feb 2025   Prob (F-statistic):               0.00
Time:                        22:36:15   Log-Likelihood:            -5.3138e+07
No. Observations:             8691158   AIC:                         1.063e+08
Df Residuals:                 8691148   BIC:                         1.063e+08
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------


In [19]:
print("Model 1: Cross-Sectional Data for Males")
regress_model(df_cross_m)

Model 1: Cross-Sectional Data for Males
                            OLS Regression Results                            
Dep. Variable:               OCCSCORE   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     9191.
Date:                Tue, 04 Feb 2025   Prob (F-statistic):               0.00
Time:                        22:36:36   Log-Likelihood:            -4.8742e+07
No. Observations:             7988379   AIC:                         9.748e+07
Df Residuals:                 7988369   BIC:                         9.748e+07
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------

In [20]:
print("Model 1: Cross-Sectional Data for Females")
regress_model(df_cross_f)

Model 1: Cross-Sectional Data for Females
                            OLS Regression Results                            
Dep. Variable:               OCCSCORE   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     1265.
Date:                Tue, 04 Feb 2025   Prob (F-statistic):               0.00
Time:                        22:36:37   Log-Likelihood:            -4.2086e+06
No. Observations:              702779   AIC:                         8.417e+06
Df Residuals:                  702769   BIC:                         8.417e+06
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------

## Model 2: Repeated Cross-Section Model
### Pooling 1900, 1910, 1920 samples with $\mu_m$ cohort fixed effects.

In [21]:
print("Model 2: Repeated Cross-Sectional Data")
regress_model(df_rep_cross)

Model 2: Repeated Cross-Sectional Data
                            OLS Regression Results                            
Dep. Variable:               OCCSCORE   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     8502.
Date:                Tue, 04 Feb 2025   Prob (F-statistic):               0.00
Time:                        22:36:45   Log-Likelihood:            -5.3137e+07
No. Observations:             8691158   AIC:                         1.063e+08
Df Residuals:                 8691148   BIC:                         1.063e+08
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------

In [22]:
print("Model 2: Repeated Cross-Sectional Data for Males")
regress_model(df_rep_cross_m)

Model 2: Repeated Cross-Sectional Data for Males
                            OLS Regression Results                            
Dep. Variable:               OCCSCORE   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     9176.
Date:                Tue, 04 Feb 2025   Prob (F-statistic):               0.00
Time:                        22:36:53   Log-Likelihood:            -4.8741e+07
No. Observations:             7988379   AIC:                         9.748e+07
Df Residuals:                 7988369   BIC:                         9.748e+07
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------

In [23]:
print("Model 2: Repeated Cross-Sectional Data for Females")
regress_model(df_rep_cross_f)

Model 2: Repeated Cross-Sectional Data for Females
                            OLS Regression Results                            
Dep. Variable:               OCCSCORE   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     1251.
Date:                Tue, 04 Feb 2025   Prob (F-statistic):               0.00
Time:                        22:36:54   Log-Likelihood:            -4.2087e+06
No. Observations:              702779   AIC:                         8.417e+06
Df Residuals:                  702769   BIC:                         8.417e+06
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------

## Model 3: Long-term Migrant Model
### Linked panel data controlling for individual characteristics.

In [24]:
print("Model 3: Long-Term Panel Data")
regress_model(df_panel)

Model 3: Long-Term Panel Data
                            OLS Regression Results                            
Dep. Variable:               OCCSCORE   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     5568.
Date:                Tue, 04 Feb 2025   Prob (F-statistic):               0.00
Time:                        22:37:01   Log-Likelihood:            -4.0809e+07
No. Observations:             6686493   AIC:                         8.162e+07
Df Residuals:                 6686483   BIC:                         8.162e+07
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------


In [25]:
print("Model 3: Long-Term Panel Data for Males")
regress_model(df_panel_m)

Model 3: Long-Term Panel Data for Males
                            OLS Regression Results                            
Dep. Variable:               OCCSCORE   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     5497.
Date:                Tue, 04 Feb 2025   Prob (F-statistic):               0.00
Time:                        22:37:08   Log-Likelihood:            -4.0156e+07
No. Observations:             6577344   AIC:                         8.031e+07
Df Residuals:                 6577334   BIC:                         8.031e+07
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------

In [26]:
print("Model 3: Long-Term Panel Data for Females")
regress_model(df_panel_f)

Model 3: Long-Term Panel Data for Females
                            OLS Regression Results                            
Dep. Variable:               OCCSCORE   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     263.5
Date:                Tue, 04 Feb 2025   Prob (F-statistic):               0.00
Time:                        22:37:09   Log-Likelihood:            -6.4531e+05
No. Observations:              109149   AIC:                         1.291e+06
Df Residuals:                  109139   BIC:                         1.291e+06
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------

# Conclusions

## Comparison to "A Nation of Immigrants" (2014) 

Comparing our results to Table 4 of "A Nation of Immigrants" (2014), we see widely different outcomes. In their regressions, the Cross-Section model showed that the incomes of immigrants converged to those of native-born workers, as the coefficients shifted from negative to positive the longer immigrants stayed in the U.S. However, when controlling for cohort fixed effects, the magnitude of this convergence decreased significantly, though the same pattern remained. Then, when linked panel data was used, the data showed no statistically significant convergence from negative to positive. In fact, the coefficients were all positive and of small magnitudes for all length-of-stay categorical variables, suggesting that, after controlling for all effects and individual characteristics, there was actually a slight premium to being an immigrant in the workforce, regardless of how long they had been in the U.S.

On the other hand, our Cross-Section data shows statistically significant negative coefficients for 0 to 5 years, 11 to 20 years, and 30+ years, but positive coefficients for 6 to 10 years and 11 to 20 years. This suggests that immigrants initially face a penalty upon arrival but, during their middle years, earn more than native-born workers—until the later stages of their careers, when they earn less again. A possible explanation is that immigrants take low-paying jobs when they first arrive but, due to a stronger work ethic compared to the average native-born worker, they move into higher-paying occupations. However, later in their careers, cultural differences and social norms may make native-born citizens more likely to receive promotions, causing immigrants' earnings to decline relative to their peers. Once we introduce controls in Model 2 and Model 3, we see similar patterns but without the initial penalty. In both models, we observe positive coefficients at the start (0-5 years) that gradually become more negative, turning negative at 21-30 years. The only minor difference is that the coefficient for 6-10 years is slightly higher than for both 0-5 years and 11-20 years, though by a very small margin. One interpretation of this is that the strong work ethic of immigrants helps them secure better-paying jobs early on, but as they move up the occupational ladder, social norms and cultural differences limit their earning potential later in life. These findings are significantly different from those in A Nation of Immigrants (2014).

It is worth noting, however, that the data used here is entirely different. Abramitzky, Boustan, and Eriksson had access to very limited data, covering only men and a minimal subset of men at that—mostly in Northern states. In contrast, our dataset includes both men and women from across the country, allowing for a broader and more comprehensive analysis.

## Comparison between Males and Females

Now, let's compare estimates between males and females, since we have access to these subsets of data.

We see that for males, the data is actually quite similar to the overall data. The relative magnitudes and signs of the coefficients are more or less the same, suggesting that they may tell the same story as the overall results. However, for women, we see a very different pattern. In all of the models for women, the coefficients are very statistically significant and very negative. In fact, there isn't a single positive coefficient for the length-of-stay variables in any of the models. This suggests that no matter how long an immigrant women stays in the US, they will not be in occupations that pay as well as native-born women. This is a very interesting result and suggests that there may be a gendered aspect to the occupational earnings. It is especially interesting considering that we see this for our models whether we are controlling for cohort fixed effects or individual characteristics. It is worth nothing though that in these samples, less than 2% of the data is of females. This may suggest that the data is not representative of the population as a whole and that the results may not be generalizable to the entire population.